**Autor: German Bertachini - 58750**

# Redes Neuronales - Trabajo Practico N1

## Problema de regresión

Se cuenta con el siguiente [dataset](https://archive.ics.uci.edu/ml/datasets/Physicochemical+Properties+of+Protein+Tertiary+Structure) para formar para construir un algoritmo predictor.


### Librerías a utilizar

In [19]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, roc_auc_score, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

#### Armamos nuestro dataframe usando Pandas

In [2]:
#features
features = ['F1','F2','F3','F4','F5','F6','F7','F8','F9']
RMSD =['RMSD']
cols= RMSD + features
data_frame= pd.read_csv('CASP.csv', sep=',')
df_RMSD=data_frame["RMSD"]
data_frame.drop(columns="RMSD", inplace=True)

Se analiza el data frame obtenido buscando patrones. Contamos con 1483 muestras de de un subconjunto de 10 clases de proteneinas, las cuales se tienen asociadas 8 parametros.

In [4]:
print(data_frame.shape)

(45730, 9)


In [106]:
df_RMSD.head()

0    17.284
1     6.021
2     9.275
3    15.851
4     7.962
Name: RMSD, dtype: float64

### Normalizamos nuestro dataset
Resulta obligatorio normalizar nuestro dataset al tratarse de un caso de regresión.

In [6]:
data_frame=((data_frame-data_frame.mean())/data_frame.std())/data_frame.max()

In [23]:
normalized_df=pd.concat([data_frame,df_RMSD],axis=1)

In [8]:
normalized_df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,RMSD
0,0.000023,0.000057,0.416976,0.002867,1.634555e-07,0.001664,0.000001,0.001620,-0.022660,17.284
1,-0.000023,-0.000062,-1.108272,-0.002448,-1.830124e-07,-0.001395,-0.000003,-0.001567,0.012166,6.021
2,-0.000013,-0.000058,-2.173551,-0.001769,-9.481921e-08,-0.001524,-0.000005,-0.002072,0.012968,9.275
3,-0.000009,-0.000029,-0.585817,-0.001742,-5.113615e-08,-0.000864,-0.000004,0.000001,0.013733,15.851
4,-0.000015,-0.000057,-1.915627,-0.002495,-1.125187e-07,-0.001220,-0.000006,-0.001465,0.016303,7.962


Distribucion probabilistica de los features.

In [22]:
normalized_df.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,RMSD
count,4.573000e+04,4.573000e+04,4.573000e+04,4.573000e+04,4.573000e+04,4.573000e+04,4.573000e+04,4.573000e+04,4.573000e+04,45730.000000
mean,-5.237197e-19,1.695032e-18,-2.632733e-14,2.600930e-17,1.949226e-21,1.541391e-17,-8.053947e-20,-1.334235e-19,-1.342439e-15,7.748528
std,2.497821e-05,6.530825e-05,1.731032e+00,2.707701e-03,1.827482e-07,1.671101e-03,9.438577e-06,2.857143e-03,1.808289e-02,6.118312
min,-4.603729e-05,-1.165774e-04,-5.777607e+00,-4.552277e-03,-3.398146e-07,-2.713603e-03,-1.888950e-05,-3.538973e-03,-5.835043e-02,0.000000
25%,-1.806468e-05,-4.630873e-05,-1.201587e+00,-1.950646e-03,-1.343655e-07,-1.214678e-03,-3.903280e-06,-1.971155e-03,-1.239526e-02,2.305000
50%,-5.987623e-06,-1.557471e-05,-6.171133e-02,-7.695214e-04,-4.246997e-08,-4.646203e-04,-7.082148e-07,-8.079355e-04,2.345538e-03,5.030000
75%,1.387698e-05,3.429898e-05,1.114775e+00,1.473142e-03,1.045293e-07,8.553847e-04,3.098429e-06,1.063331e-03,1.314582e-02,13.379000
max,1.856579e-04,5.483354e-04,7.578017e+00,1.298644e-02,1.329605e-06,1.080904e-02,4.827220e-04,1.416220e-02,6.283074e-02,20.999000


### Dividimos nuestro data set
Asignamos un 70% para train y un 15% para test y valid.

In [161]:
df_train,df_aux=train_test_split(normalized_df, test_size=0.3, shuffle=True) 
df_test,df_val=train_test_split(df_aux, test_size=0.5, shuffle=True) 
X_train=df_train.drop(columns="RMSD").values
X_test=df_test.drop(columns="RMSD").values
y=df_RMSD.values
y_train = df_train.RMSD.values
y_test = df_test.RMSD.values

In [111]:
y

array([17.284,  6.021,  9.275, ..., 10.356,  9.791, 18.827])

In [162]:
LR = LinearRegression()
LR.fit(X_train,y_train)
y_pred=LR.predict(X_test)

### Predictor sin optimizaciones

In [163]:
coef=LR.coef_
MSE=mean_squared_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Los coeficientes obtenidos son: \n", coef)
print("MSE  \n%.2f" % MSE)
print("Score \n%.2f" % score)


Los coeficientes obtenidos son: 
 [ 2.61004768e+05  3.29132909e+04  6.35810526e-01 -2.18084042e+03
 -1.37613774e+07 -1.05712454e+03 -2.60069524e+04  2.95431804e+02
 -3.63698398e+01]
MSE  
27.42
Score 
0.28


Los resultados obtenidos ($MSE=27.42$ y un $score=0.28$) son malos, de ahi la necesidad de optimizacion usando features polinomiales para una mejor aproximacion.

### Features polinomiales
Defino los features polinomiales a usar y busco cual es el que mejor resultado dado mi data frame.

In [175]:
give_me_my_best_poly_order()

Para un orden 1
 MSE: 26.883806 
 Score 0.284245 
 
Para un orden 2
 MSE: 24.197869 
 Score 0.355756 
 
Para un orden 3
 MSE: 23.430014 
 Score 0.376199 
 
Para un orden 4
 MSE: 34.945097 
 Score 0.069621 
 
Para un orden 5
 MSE: 52.213734 
 Score -0.390139 
 


In [174]:
def give_me_my_best_poly_order():
    for i in range(1,6):      
        poly=PolynomialFeatures(i)
        poly_feat=poly.fit_transform(normalized_df.drop(columns="RMSD").values)
        #we split and shuffle the data set in every iteration
        X_train,X_aux,y_train,y_aux=train_test_split(poly_feat,y, test_size=0.3, shuffle=False, random_state=42) 
        X_test,X_val,y_test, y_val=train_test_split(X_aux,y_aux, test_size=0.5, shuffle=False, random_state=42) 
        LR = LinearRegression()
        LR.fit(X_train, y_train)
        y_pred=LR.predict(X_val)
        print("Para un orden %i\n MSE: %f \n Score %f \n " % (i, mean_squared_error(y_val,y_pred), r2_score(y_val,y_pred)))
 
        

### Test
Elegimos features polinomiales de orden 3 por ser los que mejor optimizan el MSE y el score. Aplicamos dichos features en el test data set

In [178]:
        poly=PolynomialFeatures(3)
        poly_feat=poly.fit_transform(normalized_df.drop(columns="RMSD").values)
        X_train,X_aux,y_train,y_aux=train_test_split(poly_feat,y, test_size=0.3, shuffle=False, random_state=42) 
        X_test,X_val,y_test, y_val=train_test_split(X_aux,y_aux, test_size=0.5, shuffle= False, random_state=42) 
        LR = LinearRegression()
        LR.fit(X_train, y_train)
        y_pred=LR.predict(X_test)
        print("Para un orden %i\n MSE: %f \n Score %f \n " % (3, mean_squared_error(y_test,y_pred), r2_score(y_test,y_pred)))

Para un orden 3
 MSE: 22.256534 
 Score 0.424631 
 


Se obtiene un $MSE=22.25$ y un $score=0.42$, mejorando sustancialmente respecto de la primera iteracion sin features polinomiales.
